In [1]:
# Ignore warning message
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Import python libraries
import math, json
import numpy as np
from numpy import pi
from numpy.linalg import eigvals
from numpy.fft import fft, rfft
from scipy.interpolate import CubicSpline
from scipy.optimize import curve_fit, minimize
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import colors

In [3]:
# Import Qiskit libraries
import qiskit
import qiskit_aer.noise as noise
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, IBMQ, Aer, transpile, execute
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit.result import marginal_counts

In [4]:
# Load IBMQ account
provider = IBMQ.load_account()

In [5]:
# Problem: Max-cut problem

import networkx as nx
from scipy.optimize import minimize

def maxcut_obj(x, G):
    
    obj = 0
    for i, j in G.edges():
        if x[i] != x[j]:
            obj -= 1
            
    return obj

def compute_expectation(counts, G):
    
    avg = 0
    sum_count = 0
    for bitstring, count in counts.items():
        
        obj = maxcut_obj(bitstring[::-1], G)
        avg += obj * count
        sum_count += count
        
    return avg/sum_count

def create_qaoa_circ(G, theta):
    
    nqubits = len(G.nodes())
    p = len(theta)//2  # number of alternating unitaries
    qc = QuantumCircuit(nqubits)
    
    beta = theta[:p]
    gamma = theta[p:]
    
    # initial_state
    for i in range(0, nqubits):
        qc.h(i)
    
    for irep in range(0, p):
        
        # problem unitary
        for pair in list(G.edges()):
            qc.rzz(2 * gamma[irep], pair[0], pair[1])

        # mixer unitary
        for i in range(0, nqubits):
            qc.rx(2 * beta[irep], i)
            
    qc.measure_all()
        
    return qc

def get_expectation(G, shots=512):
    
    backend = Aer.get_backend('qasm_simulator')
    backend.shots = shots
    
    def execute_circ(theta):
        
        qc = create_qaoa_circ(G, theta)
        counts = backend.run(qc, seed_simulator=10, nshots=512).result().get_counts()
        
        return compute_expectation(counts, G)
    
    return execute_circ

G = nx.Graph()
G.add_nodes_from([0, 1, 2, 3])
G.add_edges_from([(0, 1), (1, 2), (2, 3), (3, 0)])

expectation = get_expectation(G)
result = minimize(expectation, [1.0, 1.0], method='Nelder-Mead', tol=1e-3)
result

 final_simplex: (array([[1.17099609, 0.40429687],
       [1.17169132, 0.4045845 ],
       [1.17166352, 0.40435352]]), array([-2.9921875, -2.9921875, -2.9921875]))
           fun: -2.9921875
       message: 'Optimization terminated successfully.'
          nfev: 51
           nit: 22
        status: 0
       success: True
             x: array([1.17099609, 0.40429687])

In [6]:
# Problem: Rosenberg function

from numpy import atleast_1d, eye, argmin, zeros, shape, squeeze, asarray, sqrt, Inf, asfarray
import numpy as np
from scipy.optimize import minimize

def rosen(x):
    
    x = np.array(x)
    r = np.sum(100.0 * a * (x[1:] - x[:-1]**2.0)**2.0 + (1 - x[:-1])**2.0, axis=0)
    
    return r

def callback(x):
    
    global Nfeval
    print('     {0:4d}  {1:3.6f}  {2:3.6f}  {3:3.6f}  {4:3.6f}  {5:3.6f}'.format(Nfeval, x[0], x[1], x[2], x[3], rosen(x)))
    Nfeval += 1

print('{0:4s} {1:9s} {2:9s} {3:9s} {4:9s}  {5:9s}'.format('Iteration', ' X1', ' X2', ' X3', ' X4', 'f(X)'))

Nfeval = 1
a = 1
x0 = [1.3, 0.7, 0.9, 1.1]
result = minimize(rosen, x0, method='Nelder-Mead', tol=1e-3, callback=callback)
result

Iteration  X1        X2        X3        X4        f(X)     
        1  1.170000  0.726250  0.933750  1.141250  65.172520
        2  1.170000  0.726250  0.933750  1.141250  65.172520
        3  1.170000  0.726250  0.933750  1.141250  65.172520
        4  1.080625  0.803359  0.909141  1.107734  28.195354
        5  0.916094  0.784629  1.005996  1.113535  16.619523
        6  0.916094  0.784629  1.005996  1.113535  16.619523
        7  0.855664  0.810913  0.947681  1.161499  16.033327
        8  0.855664  0.810913  0.947681  1.161499  16.033327
        9  0.971858  0.810172  0.941850  1.125892  15.697290
       10  0.827151  0.837113  0.972495  1.137940  13.473976
       11  0.835774  0.821951  0.969411  1.102080  12.850335
       12  0.835774  0.821951  0.969411  1.102080  12.850335
       13  0.902912  0.858406  0.967798  1.057195  7.003432
       14  0.902912  0.858406  0.967798  1.057195  7.003432
       15  0.902912  0.858406  0.967798  1.057195  7.003432
       16  0.902912  0.8584

 final_simplex: (array([[0.99414703, 0.98830096, 0.97687033, 0.95476054],
       [0.99452298, 0.98823825, 0.97699642, 0.95436488],
       [0.99442256, 0.98835564, 0.9770429 , 0.95376162],
       [0.99383055, 0.98771587, 0.97681849, 0.95460605],
       [0.99388788, 0.98854472, 0.97736131, 0.9538601 ]]), array([0.00073142, 0.0007847 , 0.00079713, 0.00089774, 0.00092566]))
           fun: 0.0007314240726394858
       message: 'Optimization terminated successfully.'
          nfev: 109
           nit: 60
        status: 0
       success: True
             x: array([0.99414703, 0.98830096, 0.97687033, 0.95476054])

In [8]:
# Problem: Transverse-field Ising model

# Set up parameters
Nc = 5  # number of spins: N (= 2, 3, 4, 5)
g = 0.4  # dimensionless Ising coupling: J/h (= 0.2, 0.4, 0.6, 0.8)
eta = 0.3  # dimensionless broadening: \eta/h (= 0.02, 0.1, 0.2, 0.3)
dw = 0.25*eta  # dimensionless unit of discrete frequency: \delta\omega / h
L = 2*math.ceil(7.0/dw)  # count of Fourier components: L
Le = 100
dt = 2*pi/L/dw  # dimensionless unit of discrete time: h \delta t
M = 35  # Trotter depth cutoff: M
shots = 1024  # count of measurement shots

# Define the identity and Pauli matrices
s0 = np.matrix([[1,0],[0,1]])
s1 = np.matrix([[0,1],[1,0]])
s2 = np.matrix([[0,-1j],[1j,0]])
s3 = np.matrix([[1,0],[0,-1]])

# Define the model Hamitonian
Ham = np.kron(s1,s0) + np.kron(s0,s1) + g * np.kron(s3,s3)
if Nc >= 3:
    for j in range(3,Nc+1):  
        Ide = s0
        for k in range(j-3):
            Ide = np.kron(Ide,s0)
        Ham = np.kron(Ham,s0) + np.kron(np.kron(Ide,s0),s1) + g * np.kron(np.kron(Ide,s3),s3)

# Calculate the exact energy gap
evals = eigvals(Ham)
idx = evals.argsort()[::1]
evals = evals[idx]
ExactGap = evals[1] - evals[0]

# set the initial guess of energy gap
IniGap = 2*(1-(1-1/Nc)*g)

# Define the bit string (00...0) as a reference for thr initial state
s = []
for j in range (Nc):
    s.append('0')
str1 = ""
InQ = str1.join(s)

# Define discrete frequencies
frequency = [(j-L/2)*dw for j in range(L+1)]
with open('data_frequency', 'w') as f:
    f.write(json.dumps(frequency))
        
# Define the objective function for curve fitting of the Lorentzian function
def objective (x, a0, a1, a2):
    return a0 + a1 / ((x - a2)**2 + eta**2)

# Print preliminary data
print("N = %s, J/h = %.2f, M = %s" % (Nc,g,M))
print("Initial gap = %.6f" % IniGap)
print("Exact gap = %.6f" % ExactGap)

N = 5, J/h = 0.40, M = 35
Initial gap = 1.360000
Exact gap = 1.340771


In [10]:
# Simulation type: ideal (noiseless)
def InvPeak_Ideal(theta):
    
    global ExactGap, IniGap
    
    # number of alternating unitaries
    P = len(theta)//2
    beta = theta[:P]
    gamma = theta[P:]
    
    # Loop for causal (i=0) and anti-causal (i=1) processes
    for i in range (2):

        # Initialize array
        propagator = []
    
        # Loop for Fourier components
        for j in range (L):

            # Initialize a set of quantum/classical registers
            qreg = QuantumRegister(Nc, 'q')
            creg = ClassicalRegister(Nc, 'c')
            qc = QuantumCircuit(qreg, creg)
            
            # Set up the unitary for trial state
            for p in range (P):
    
                for n in range (Nc-1):
                    qc.rzz(gamma[p]*pi, qreg[n], qreg[n+1])
    
                for n in range (Nc):
                    qc.ry(beta[p]*pi, qreg[n])
    
            # Time evolution: Iterate Trotter steps
            for m in range (M):

                # Apply a sequence of unitary gates for time evolution
                for n in range (Nc):
                    qc.rx(-2*j*(-1)**i*dt/M, qreg[n])
                
                for n in range (Nc-1):
                    qc.rzz(-2*g*j*(-1)**i*dt/M, qreg[n], qreg[n+1])
                    
            # Set up the inverse unitary for trial state
            for p in range (P):
        
                for n in range (Nc):
                    qc.ry(-beta[P-p-1]*pi, qreg[n])
        
                for n in range (Nc-1):
                    qc.rzz(-gamma[P-p-1]*pi, qreg[n], qreg[n+1])
        
            # Measure qubits
            qc.measure(qreg, creg)
        
            # Perform quantum simulation on the IBMQ Aer simulator
            simulator = Aer.get_backend('qasm_simulator')
            result = execute(qc, simulator, shots = shots).result()
            counts = result.get_counts()
    
            # Construct a time series of propagators with each component being the probability for (00...0)
            if InQ in counts:
                propagator.append(dt*np.exp(-eta*j*dt)*counts[InQ]/shots)
            else:
                propagator.append(0)
    
            # Print the end message for each step
            if (i==1) and (j>L-10):
                print(L-1-j,end="\r")
    
        # Flatten array
        propagator = np.ravel(propagator)

        # Find the many-body spectral function via discreate fast Fourier transform
        if i == 0:
            spectral_function_0r = np.real(np.fft.ifft(propagator))*L/2.0/pi
            spectral_function_1r = np.append(spectral_function_0r[int(L/2):L], spectral_function_0r[0:int(L/2)+1])
        else:
            spectral_function_0a = np.real(np.fft.fft(propagator))/2.0/pi
            spectral_function_1a = np.append(spectral_function_0a[int(L/2):L], spectral_function_0a[0:int(L/2)+1])
    
    # Find the many-body spectral function via discreate fast Fourier transform
    spectral_function_2 = [spectral_function_1r[j] + spectral_function_1a[j] for j in range(L+1)]

    # Truncate the lists in the range [-\eta,\eta]
    frequency_truncated = [frequency[j] for j in range(math.ceil(L/2-2*eta/dw),math.ceil(L/2+1+2*eta/dw))]
    spectral_function_truncated = [spectral_function_2[j] for j in range(math.ceil(L/2-2*eta/dw),math.ceil(L/2+1+2*eta/dw))]

    # Curve fitting for the Lorentzian function
    a2 = 0
    popt, _ = curve_fit(lambda x, a0, a1: objective(x, a0, a1, a2), frequency_truncated, spectral_function_truncated)
    a0, a1 = popt

    # Subtract the zeto peak from the manu-body spectral function
    spectral_function = [spectral_function_2[j] - a1 / (frequency[j]**2 + eta**2) for j in range(L+1)]
    
    # Truncate the lists in the range [IniGap-\eta,IniGap+\eta]
    frequency_truncated = [frequency[j] for j in range(math.ceil(L/2+(IniGap-2*eta)/dw),math.ceil(L/2+1+(IniGap+2*eta)/dw))]
    spectral_function_truncated = [spectral_function[j] for j in range(math.ceil(L/2+(IniGap-2*eta)/dw),math.ceil(L/2+1+(IniGap+2*eta)/dw))]
    
    # Curve fitting for the Lorentzian function
    popt, _ = curve_fit(objective, frequency_truncated, spectral_function_truncated)
    a0, a1, a2 = popt
    
    return -a1/eta**2, a2, abs(a2 - ExactGap)/ExactGap, spectral_function

# Define wrapper functions
def InvPeak_Ideal_1(theta):
    obj1, obj2, obj3, obj4 = InvPeak_Ideal(theta)
    return obj1

def InvPeak_Ideal_2(theta):
    obj1, obj2, obj3, obj4 = InvPeak_Ideal(theta)
    return obj2

def InvPeak_Ideal_3(theta):
    obj1, obj2, obj3, obj4 = InvPeak_Ideal(theta)
    return obj3

def InvPeak_Ideal_4(theta):
    obj1, obj2, obj3, obj4 = InvPeak_Ideal(theta)
    return obj4

# Define callback function
def callback (theta):
    
    global data, Nfeval, theta0
    Nfeval += 1
    P0 = len(theta0)//2
    beta0 = theta0[:P0]
    gamma0 = theta0[P0:]
    P = len(theta)//2
    beta = theta[:P]
    gamma = theta[P:]
    peak = -InvPeak_Ideal_1(theta)
    gap_est = InvPeak_Ideal_2(theta)
    gap_err = InvPeak_Ideal_3(theta)
    spectral_function = InvPeak_Ideal_4(theta)
    
    # Print/save data
    print('     {0:4d}  {1:3.6f}   {2:3.6f}   {3:3.6f}   {4:3.6f}   {5:3.6f}'\
          .format(Nfeval, beta[0], gamma[0], peak, gap_est, gap_err))
    data.append([Nfeval, beta[0], gamma[0], peak, gap_est, gap_err])
        
    with open('data_spectral_ideal_N%s_g%.2f_eta%.2f_M%s_beta%.2f_gamma%.2f_NM%s' % 
              (int(Nc),g,eta,int(M),beta0[0],gamma0[0],int(Nfeval)), 'w') as f:
        f.write(json.dumps(spectral_function))

In [ ]:
# Print the start message
print("Simulation type: Ideal")
print("Optimization method: Nelder-Mead")

# Set parameters
data = []
Nfeval = 0
theta0 = [0.5, 0.1]
P0 = len(theta0)//2
beta0 = theta0[:P0]
gamma0 = theta0[P0:]
bounds = ((0,2),(0,2))

# Save data
print('{0:4s} {1:9s}   {2:9s}  {3:9s}  {4:9s}  {5:9s}'\
      .format('Iteration',' beta/pi','gamma/pi','height','gap_est','gap_error'))

# Estimate energy gap
result = minimize(InvPeak_Ideal_1, theta0, method='Nelder-Mead', tol=1e-2, bounds=bounds, callback=callback)

# Save data
with open('data_iteration_ideal_N%s_g%.2f_eta%.2f_M%s_beta%.2f_gamma%.2f_NM' % 
          (int(Nc),g,eta,int(M),beta0[0],gamma0[0]), 'w') as f:
    f.write(json.dumps(data))
            
# Print the end message
print(result)

Simulation type: Ideal
Optimization method: Nelder-Mead
Iteration  beta/pi    gamma/pi   height     gap_est    gap_error
        1  0.525000   0.100000   0.005388   1.338774   0.038006
        2  0.525000   0.100000   -0.002891   0.717938   0.462566
        3  0.528125   0.101250   -0.003630   2.038287   0.010557


In [44]:
# Build noise model from backend calibration data
backend = provider.get_backend("ibmq_manila")
noise_model = NoiseModel.from_backend(backend)
coupling_map = backend.configuration().coupling_map
basis_gates = noise_model.basis_gates

# Simulation type: Device noise model
def InvPeak_Noisy(theta):
    
    global ExactGap, IniGap
    
    # number of alternating unitaries
    P = len(theta)//2
    beta = theta[:P]
    gamma = theta[P:]
    
    # Loop for causal (i=0) and anti-causal (i=1) processes
    for i in range (2):

        # Print the message
        #print("...running")
    
        # Initialize array
        propagator = []
    
        # Loop for Fourier components
        for j in range (L):

            # Initialize a set of quantum/classical registers
            qreg = QuantumRegister(Nc, 'q')
            creg = ClassicalRegister(Nc, 'c')
            qc = QuantumCircuit(qreg, creg)
            
            # Set up input state in QAOA
            for p in range (P):
    
                for n in range (Nc-1):
                    qc.rzz(gamma[p]*pi, qreg[n], qreg[n+1])
    
                for n in range (Nc):
                    qc.ry(beta[p]*pi, qreg[n])
    
            # Time evolution: Iterate Trotter steps
            for m in range (M):

                # Apply a sequence of unitary gates for time evolution
                for n in range (Nc):
                    qc.rx(-2*j*(-1)**i*dt/M, qreg[n])
                
                for n in range (Nc-1):
                    qc.rzz(-2*g*j*(-1)**i*dt/M, qreg[n], qreg[n+1])
                    
            # Set up the state to compenstate input in QAOA
            for p in range (P):
        
                for n in range (Nc):
                    qc.ry(-beta[P-p-1]*pi, qreg[n])
        
                for n in range (Nc-1):
                    qc.rzz(-gamma[P-p-1]*pi, qreg[n], qreg[n+1])
        
            # Measure qubits
            qc.measure(qreg, creg)
        
            # Perform quantum simulation on the IBMQ Aer simulator
            simulator = Aer.get_backend('qasm_simulator')
            qc_trans = transpile(qc, backend, optimization_level = 1, initial_layout = [k for k in range(Nc)])
            result = execute(qc_trans, simulator, noise_model = noise_model, coupling_map = coupling_map,\
                         basis_gates = basis_gates, shots = shots).result()    
            counts = result.get_counts()
    
            # Construct a time series of propagators with each component being the probability for (00...0)
            if InQ in counts:
                propagator.append(dt*np.exp(-eta*j*dt)*counts[InQ]/shots)
            else:
                propagator.append(0)
    
            # Print the end message for each step
            print(j+1, end="\r")
    
        # Flatten array
        propagator = np.ravel(propagator)

        # Find the many-body spectral function via discreate fast Fourier transform
        if i == 0:
            spectral_function_0r = np.real(np.fft.ifft(propagator))*L/2.0/pi
            spectral_function_1r = np.append(spectral_function_0r[int(L/2):L], spectral_function_0r[0:int(L/2)+1])
        else:
            spectral_function_0a = np.real(np.fft.fft(propagator))/2.0/pi
            spectral_function_1a = np.append(spectral_function_0a[int(L/2):L], spectral_function_0a[0:int(L/2)+1])
    
    # Find the many-body spectral function via discreate fast Fourier transform
    spectral_function_2 = [spectral_function_1r[j] + spectral_function_1a[j] for j in range(L+1)]

    # Truncate the lists in the range [-\eta,\eta]
    frequency_truncated = [frequency[j] for j in range(math.ceil(L/2-2*eta/dw),math.ceil(L/2+1+2*eta/dw))]
    spectral_function_truncated = [spectral_function_2[j] for j in range(math.ceil(L/2-2*eta/dw),math.ceil(L/2+1+2*eta/dw))]

    # Curve fitting for the Lorentzian function
    a2 = 0
    popt, _ = curve_fit(lambda x, a0, a1: objective(x, a0, a1, a2), frequency_truncated, spectral_function_truncated)
    a0, a1 = popt
    
    # Subtract the zeto peak from the manu-body spectral function
    spectral_function = [spectral_function_2[j] - a1 / (frequency[j]**2 + eta**2) for j in range(L+1)]
    
    # Truncate the lists in the range [IniGap-\eta,IniGap+\eta]
    frequency_truncated = [frequency[j] for j in range(math.ceil(L/2+(IniGap-2*eta)/dw),math.ceil(L/2+1+(IniGap+2*eta)/dw))]
    spectral_function_truncated = [spectral_function[j] for j in range(math.ceil(L/2+(IniGap-2*eta)/dw),math.ceil(L/2+1+(IniGap+2*eta)/dw))]
    
    # Curve fitting for the Lorentzian function
    popt, _ = curve_fit(objective, frequency_truncated, spectral_function_truncated)
    a0, a1, a2 = popt
    
    return -a1/eta**2, a2, abs(a2 - ExactGap)/ExactGap, spectral_function

# Define wrapper functions
def InvPeak_Noisy_1(theta):
    obj1, obj2, obj3, obj4 = InvPeak_Noisy(theta)
    return obj1

def InvPeak_Noisy_2(theta):
    obj1, obj2, obj3, obj4 = InvPeak_Noisy(theta)
    return obj2

def InvPeak_Noisy_3(theta):
    obj1, obj2, obj3, obj4 = InvPeak_Noisy(theta)
    return obj3

def InvPeak_Noisy_4(theta):
    obj1, obj2, obj3, obj4 = InvPeak_Noisy(theta)
    return obj4

# Define callback function
def callback (theta):
    
    global data, Nfeval, theta0
    Nfeval += 1
    P0 = len(theta0)//2
    beta0 = theta0[:P0]
    gamma0 = theta0[P0:]
    P = len(theta)//2
    beta = theta[:P]
    gamma = theta[P:]
    peak = -InvPeak_Noisy_1(theta)
    gap_est = InvPeak_Noisy_2(theta)
    gap_err = InvPeak_Noisy_3(theta)
    spectral_function = InvPeak_Noisy_4(theta)
    
    # Print/save data
    print('     {0:4d}  {1:3.6f}   {2:3.6f}   {3:3.6f}   {4:3.6f}   {5:3.6f}'\
          .format(Nfeval, beta[0], gamma[0], peak, gap_est, gap_err))
    data.append([Nfeval, beta[0], gamma[0], peak, gap_est, gap_err])
        
    with open('data_spectral_noisy_N%s_g%.2f_eta%.2f_M%s_beta%.2f_gamma%.2f_NM%s' % 
              (int(Nc),g,eta,int(M),beta0[0],gamma0[0],int(Nfeval)), 'w') as f:
        f.write(json.dumps(spectral_function))

In [45]:
# Print the start message
print("Simulation type: Device noise model")
print("Optimization method: Nelder-Mead")

# Set parameters
data = []
Nfeval = 0
theta0 = [0.5, 0.1]
P0 = len(theta0)//2
beta0 = theta0[:P0]
gamma0 = theta0[P0:]
bounds = ((0,2),(0,2))

# Save data
print('{0:4s} {1:9s}   {2:9s}  {3:9s}  {4:9s}  {5:9s}'\
      .format('Iteration',' beta/pi','gamma/pi','height','gap_est','gap_error'))

# Estimate energy gap
result = minimize(InvPeak_Noisy_1, theta0, method='Nelder-Mead', tol=1e-2, bounds=bounds, callback=callback)

# Save data
with open('data_iteration_noisy_N%s_g%.2f_eta%.2f_M%s_beta%.2f_gamma%.2f_NM' % 
          (int(Nc),g,eta,int(M),beta0[0],gamma0[0]), 'w') as f:
    f.write(json.dumps(data))
            
# Print the end message
print(result)

Simulation type: Device noise model
Optimization method: Nelder-Mead
Iteration  beta/pi    gamma/pi   height     gap_est    gap_error
        1  0.450000   0.107500   0.004593   1.089513   0.125573
        2  0.450000   0.107500   0.011872   1.120592   0.185201
        3  0.350000   0.120000   0.034459   1.272770   0.071126
        4  0.300000   0.116250   0.043917   1.298224   0.034478
        5  0.300000   0.116250   0.043129   1.289023   0.041424
        6  0.300000   0.121250   0.044644   1.290256   0.037986
        7  0.250000   0.123750   0.045917   1.329466   0.001605
        8  0.262500   0.125625   0.045635   1.328710   0.019873
        9  0.212500   0.128125   0.041987   1.376184   0.022868
       10  0.212500   0.128125   0.043466   1.368076   0.014065
       11  0.212500   0.128125   0.045427   1.366853   0.022361
       12  0.212500   0.128125   0.045182   1.368701   0.013529
       13  0.212500   0.128125   0.043497   1.352449   0.026645
       14  0.212500   0.128125   0